In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import pickle

from tqdm import tqdm
from multiprocessing import Pool, cpu_count

In [2]:
from gluonts.dataset.multivariate_grouper import MultivariateGrouper
from gluonts.dataset.repository.datasets import dataset_recipes, get_dataset
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import MultivariateEvaluator
from diffusers import (
    PNDMScheduler,
    DDIMScheduler,
    DPMSolverMultistepScheduler,
    KDPM2DiscreteScheduler,
    DEISMultistepScheduler,
)

from pts.model.time_grad import TimeGradEstimator
from pts.dataset.repository.datasets import dataset_recipes

from utils import omit_points, interpolate_np_array

/home/jupyter/.local/lib/python3.10/site-packages/gluonts/dataset/artificial/_base.py:84: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  self.freq: BaseOffset = to_offset(freq)


In [3]:
dataset = get_dataset("solar_nips", regenerate=False)

train_grouper = MultivariateGrouper(
    max_target_dim=int(dataset.metadata.feat_static_cat[0].cardinality)
)

test_grouper = MultivariateGrouper(
    num_test_dates=int(len(dataset.test) / len(dataset.train)),
    max_target_dim=int(dataset.metadata.feat_static_cat[0].cardinality),
)

dataset_train = train_grouper(dataset.train)
dataset_test = test_grouper(dataset.test)

/home/jupyter/.local/lib/python3.10/site-packages/gluonts/dataset/common.py:262: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  return pd.Period(val, freq)


In [4]:
evaluator = MultivariateEvaluator(
    quantiles=(np.arange(20) / 20.0)[1:], target_agg_funcs={"sum": np.sum}
)

In [5]:
def plot(
    target,
    forecast,
    prediction_length,
    prediction_intervals=(50.0, 90.0),
    color="g",
    fname=None,
):
    label_prefix = ""
    rows = 4
    cols = 4
    fig, axs = plt.subplots(rows, cols, figsize=(24, 24))
    axx = axs.ravel()
    seq_len, target_dim = target.shape

    ps = [50.0] + [
        50.0 + f * c / 2.0 for c in prediction_intervals for f in [-1.0, +1.0]
    ]

    percentiles_sorted = sorted(set(ps))

    def alpha_for_percentile(p):
        return (p / 100.0) ** 0.3

    for dim in range(0, min(rows * cols, target_dim)):
        ax = axx[dim]

        target[-2 * prediction_length :][dim].plot(ax=ax)

        ps_data = [forecast.quantile(p / 100.0)[:, dim] for p in percentiles_sorted]
        i_p50 = len(percentiles_sorted) // 2

        p50_data = ps_data[i_p50]
        p50_series = pd.Series(data=p50_data, index=forecast.index)
        p50_series.plot(color=color, ls="-", label=f"{label_prefix}median", ax=ax)

        for i in range(len(percentiles_sorted) // 2):
            ptile = percentiles_sorted[i]
            alpha = alpha_for_percentile(ptile)
            ax.fill_between(
                forecast.index,
                ps_data[i],
                ps_data[-i - 1],
                facecolor=color,
                alpha=alpha,
                interpolate=True,
            )
            # Hack to create labels for the error intervals.
            # Doesn't actually plot anything, because we only pass a single data point
            pd.Series(data=p50_data[:1], index=forecast.index[:1]).plot(
                color=color,
                alpha=alpha,
                linewidth=10,
                label=f"{label_prefix}{100 - ptile * 2}%",
                ax=ax,
            )

    legend = ["observations", "median prediction"] + [
        f"{k}% prediction interval" for k in prediction_intervals
    ][::-1]
    axx[0].legend(legend, loc="upper left")

    if fname is not None:
        plt.savefig(fname, bbox_inches="tight", pad_inches=0.05)


In [6]:
scheduler = DEISMultistepScheduler(
    num_train_timesteps=150,
    beta_end=0.1,
)

In [7]:
estimator = TimeGradEstimator(
    input_size=int(dataset.metadata.feat_static_cat[0].cardinality),
    hidden_size=64,
    num_layers=2,
    dropout_rate=0.1,
    lags_seq=[1],
    scheduler=scheduler,
    num_inference_steps=149,
    prediction_length=dataset.metadata.prediction_length,
    context_length=dataset.metadata.prediction_length,
    freq=dataset.metadata.freq,
    scaling="mean",
    trainer_kwargs=dict(max_epochs=200, accelerator="gpu", devices="1")
)

/home/jupyter/.local/lib/python3.10/site-packages/gluonts/time_feature/_base.py:243: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  offset = to_offset(freq_str)


In [8]:
predictor = estimator.train(dataset_train, cache_data=True, shuffle_buffer_length=1024)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/jupyter/.local/lib/python3.10/site-packages/lightning/pytorch/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
2024-05-22 13:31:52.321965: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-22 13:31:55.825753: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type          | Params | In sizes                                                             | Out sizes        
-------------------------------------------------------

Epoch 0: |          | 50/? [00:07<00:00,  6.72it/s, v_num=15, train_loss=0.385]

Epoch 0, global step 50: 'train_loss' reached 0.38476 (best 0.38476), saving model to '/home/jupyter/work/resources/lightning_logs/version_15/checkpoints/epoch=0-step=50.ckpt' as top 1


Epoch 1: |          | 50/? [00:06<00:00,  7.97it/s, v_num=15, train_loss=0.155]

Epoch 1, global step 100: 'train_loss' reached 0.15453 (best 0.15453), saving model to '/home/jupyter/work/resources/lightning_logs/version_15/checkpoints/epoch=1-step=100.ckpt' as top 1


...

Epoch 198: |          | 50/? [00:06<00:00,  8.13it/s, v_num=15, train_loss=0.0404]

Epoch 198, global step 9950: 'train_loss' was not in top 1


Epoch 199: |          | 50/? [00:06<00:00,  8.16it/s, v_num=15, train_loss=0.0416]

Epoch 199, global step 10000: 'train_loss' was not in top 1
`Trainer.fit` stopped: `max_epochs=200` reached.


Epoch 199: |          | 50/? [00:06<00:00,  8.15it/s, v_num=15, train_loss=0.0416]


In [9]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=dataset_test, predictor=predictor, num_samples=100
)
forecasts = list(forecast_it)
targets = list(ts_it)
agg_metric, _ = evaluator(targets, forecasts, num_series=len(dataset_test))

Running evaluation: 7it [00:00, 58.25it/s]
Running evaluation: 7it [00:00, 66.72it/s]
Running evaluation: 7it [00:00, 70.20it/s]
Running evaluation: 7it [00:00, 65.56it/s]
Running evaluation: 7it [00:00, 65.72it/s]
Running evaluation: 7it [00:00, 63.88it/s]
Running evaluation: 7it [00:00, 63.29it/s]
Running evaluation: 7it [00:00, 65.00it/s]
Running evaluation: 7it [00:00, 66.85it/s]
Running evaluation: 7it [00:00, 65.02it/s]
Running evaluation: 7it [00:00, 68.68it/s]
Running evaluation: 7it [00:00, 70.25it/s]
Running evaluation: 7it [00:00, 66.68it/s]
Running evaluation: 7it [00:00, 68.54it/s]
Running evaluation: 7it [00:00, 65.28it/s]
Running evaluation: 7it [00:00, 70.84it/s]
Running evaluation: 7it [00:00, 69.58it/s]
Running evaluation: 7it [00:00, 64.63it/s]
Running evaluation: 7it [00:00, 64.15it/s]
Running evaluation: 7it [00:00, 61.15it/s]
Running evaluation: 7it [00:00, 63.26it/s]
Running evaluation: 7it [00:00, 69.66it/s]
Running evaluation: 7it [00:00, 63.63it/s]
Running eva

In [10]:
print('Timegrad on regular dataset metrics:')
print("CRPS: {}".format(agg_metric["mean_wQuantileLoss"]))
print("ND: {}".format(agg_metric["ND"]))
print("NRMSE: {}".format(agg_metric["NRMSE"]))
print("MSE: {}".format(agg_metric["MSE"]))

print('')

print("CRPS-Sum: {}".format(agg_metric["m_sum_mean_wQuantileLoss"]))
print("ND-Sum: {}".format(agg_metric["m_sum_ND"]))
print("NRMSE-Sum: {}".format(agg_metric["m_sum_NRMSE"]))
print("MSE-Sum: {}".format(agg_metric["m_sum_MSE"]))

Timegrad on regular dataset metrics:
CRPS: 0.3743160235033126
ND: 0.4681447740693562
NRMSE: 0.9333072944939037
MSE: 826.2797468362436

CRPS-Sum: 0.31375429086490336
ND-Sum: 0.3635174670093458
NRMSE-Sum: 0.6307043416960839
MSE-Sum: 7082239.095238095


In [11]:
with open('/home/jupyter/datasphere/project/models/solar.pkl', 'wb') as f:
    pickle.dump(predictor, f)

In [12]:
"""
# If you want to load already prepared model

transformation = estimator.create_transformation()
lightning_module = estimator.create_lightning_module()
predictor = estimator.create_predictor(transformation, lightning_module)
with open('/home/jupyter/work/resources/solar/model.pkl', 'rb') as f:
    predictor = pickle.load(f)
"""

"\n# If you want to load already prepared model\n\ntransformation = estimator.create_transformation()\nlightning_module = estimator.create_lightning_module()\npredictor = estimator.create_predictor(transformation, lightning_module)\nwith open('/home/jupyter/work/resources/solar/model.pkl', 'rb') as f:\n    predictor = pickle.load(f)\n"

In [13]:
def interpolate_points(dataset, num_points: int = 5):
    new_dataset = []
    with Pool(processes=cpu_count()) as pool:
        results = pool.starmap(interpolate_np_array, [(ts['target'], num_points) for ts in dataset])
        for i, ts in enumerate(dataset):
            mask = np.isnan(ts['target'])
            new_dataset.append({
                'target': ts['target'] * (1. - mask) + results[i] * mask,
                'start': ts['start'],
                'feat_static_cat': ts['feat_static_cat']
            })
    return new_dataset

In [14]:
crps_sum_results = pd.DataFrame(columns=range(10))

with tqdm(total=9*10) as pbar:
    for p in np.linspace(0.1, 1, 9, endpoint=False): # frac of removed points
        dataset_test_p = omit_points(dataset_test, p)
        results_p = []

        # No interpolation
        forecast_it, ts_it = make_evaluation_predictions(
            dataset=dataset_test_p, 
            predictor=predictor, 
            num_samples=100,
        )
        forecasts = list(forecast_it)
        # targets = list(ts_it)  # Do not uncomment, we need targets without NaNs, so we use ones calculated above.
        agg_metric, _ = evaluator(targets, forecasts, num_series=len(dataset_test))
        results_p.append(agg_metric['m_sum_mean_wQuantileLoss'])
        pbar.update(1)

        # Interpolation with num_points
        for num_points in range(1, 10):
            forecast_it, ts_it = make_evaluation_predictions(
                dataset=interpolate_points(dataset_test_p, num_points), 
                predictor=predictor, 
                num_samples=100,
            )
            forecasts = list(forecast_it)
            # targets = list(ts_it)  # Do not uncomment, we need targets without NaNs, so we use ones calculated above.
            agg_metric, _ = evaluator(targets, forecasts, num_series=len(dataset_test))
            results_p.append(agg_metric['m_sum_mean_wQuantileLoss'])
            pbar.update(1)

        crps_sum_results.loc[p] = results_p

  0%|          | 0/90 [00:00<?, ?it/s]
Running evaluation: 7it [00:00, 66.04it/s]

Running evaluation: 7it [00:00, 69.74it/s]

Running evaluation: 7it [00:00, 61.04it/s]

Running evaluation: 7it [00:00, 60.16it/s]

Running evaluation: 7it [00:00, 63.60it/s]

Running evaluation: 7it [00:00, 66.33it/s]

Running evaluation: 7it [00:00, 62.18it/s]

Running evaluation: 7it [00:00, 68.40it/s]

Running evaluation: 7it [00:00, 65.29it/s]

Running evaluation: 7it [00:00, 69.06it/s]

Running evaluation: 7it [00:00, 70.59it/s]

Running evaluation: 7it [00:00, 67.75it/s]

Running evaluation: 7it [00:00, 69.96it/s]

Running evaluation: 7it [00:00, 67.39it/s]

Running evaluation: 7it [00:00, 67.09it/s]

Running evaluation: 7it [00:00, 69.16it/s]

Running evaluation: 7it [00:00, 62.41it/s]

Running evaluation: 7it [00:00, 68.46it/s]

Running evaluation: 7it [00:00, 69.28it/s]

Running evaluation: 7it [00:00, 66.52it/s]

Running evaluation: 7it [00:00, 67.76it/s]

Running evaluation: 7it [00:00, 63.83

In [15]:
crps_sum_results

,0,1,2,3,4,5,6,7,8,9
0.1,0.316294,0.308476,0.326342,0.367161,0.323442,0.312021,0.319708,0.312569,0.328698,0.309832
0.2,0.365494,0.342766,0.360484,0.396763,0.355549,0.343080,0.360361,0.344270,0.356978,0.344453
0.3,0.357299,0.313618,0.352668,0.366452,0.353973,0.312520,0.356401,0.317901,0.354960,0.316115
0.4,0.331644,0.373367,0.343645,0.430531,0.346478,0.377750,0.346317,0.375730,0.347669,0.377319
0.5,0.418495,0.416832,0.336657,0.469853,0.334727,0.419248,0.334798,0.417846,0.335849,0.414912
0.6,0.346795,0.334831,0.410464,0.385944,0.408754,0.337744,0.409436,0.337253,0.408815,0.334682
0.7,0.388019,0.327894,0.371527,0.370052,0.368144,0.319977,0.368330,0.319120,0.367450,0.323415
0.8,0.400023,0.429624,0.392653,0.488320,0.387170,0.426110,0.389264,0.428022,0.390439,0.428694
0.9,1.458100,1.346979,1.178966,1.393135,1.177913,1.346834,1.181553,1.348415,1.181906,1.350060
